In [4]:
import pandas as pd

In [5]:
import numpy as np

# Constantes

In [6]:
k = 10
L0 = 1.986
m0 = 0.993
a = 1
g = 9.81

In [7]:
tolerancia = 0.5*(10**(-15)) #0,5 * 10exp(-15)

# Funciones

In [8]:
def f (x):
    return -2*k*x*(1-(L0)/np.sqrt(x**2+a**2))
def f_m (x,coef):
    return f(x)-m0*coef*g
def der_f(x):
    return ((-2*k*L0*x**2) / ((a**2+x**2)**(3/2))) - 2*k*(1-(L0/np.sqrt(a**2+x**2)))

In [9]:
def nr (p0,f,der_f,max_iter,tol,true_root = "None"):
    dif_iter = []
    output = []
    cols = ["y","deltay","deltay/y","lambda","p"]
    c = True
    i = 0
    p = p0
    while (c and (i<=max_iter)):
        old_p = p
        p = p - f(p)/der_f(p)
        output.append([old_p,np.abs(p-old_p),np.abs(p-old_p)/old_p,0,p])
        dif_iter.append(np.abs(p-old_p))
        if true_root == "None":
            c = (np.abs(p-old_p) >= tol)
        else:
            c = (np.abs(p-true_root) >= tol)
        i+=1
    df = pd.DataFrame(output)
    df.columns = cols
    return df,dif_iter
        

In [32]:
def biseccion (a,b,f,max_iter,tol,true_root = "None"):
    cols = ["a","b","f(a)","f(b)","r","deltar","deltar/r","lambda","p"]
    output=[]
    c=True
    i=0
    while (c and (i<=max_iter)):
        p = (a+b)/2
        table_entry = [a,b,f(a),f(b),0,0,0,0,p]
        
        if f(a)*f(p) < 0:
            b=p
        else:
            a=p
        
        next_p_dif = np.abs(((a+b)/2)-p)        
        
        if (true_root == "None"):
            table_entry[4] = (table_entry[1]-table_entry[0])/2
            c = (next_p_dif > tol)
        else:
            table_entry[4] = np.abs(p-true_root)
            c = (table_entry[4] > tol)
            
        table_entry[5] = np.abs(next_p_dif)
        table_entry[6] = table_entry[5]/table_entry[4]
            
        output.append(table_entry)
        i+=1       
    df = pd.DataFrame(output)
    df.columns = cols
    return df;
            

In [33]:
bisec1 = biseccion(-0.5,1,f,200,tolerancia,0)

In [75]:
bisec2 = biseccion(1.7,1.8,f,200,tolerancia,np.sqrt(L0**2-a**2))

In [38]:
nr1 = nr(der_f=der_f,f=f,p0=0.1,tol=tolerancia,true_root=0,max_iter=30)

In [39]:
nr2 = nr(der_f=der_f,f=f,p0=-1.7,tol=tolerancia,max_iter=30)